In [1]:
# install packages needed
!pip install "skm-tools[cytoscape] @ git+ssh://git@github.com/NIB-SI/skm-tools.git"
!pip install sbmlutils

  Cloning ssh://****@github.com/NIB-SI/skm-tools.git to /tmp/pip-install-r68dgifd/skm-tools_d815aad9fbb34287bbf27b969634ee31
  Running command git clone --filter=blob:none --quiet 'ssh://****@github.com/NIB-SI/skm-tools.git' /tmp/pip-install-r68dgifd/skm-tools_d815aad9fbb34287bbf27b969634ee31
  Resolved ssh://****@github.com/NIB-SI/skm-tools.git to commit af75f2cff9e0a23b92323806349707f097641cc6
  Preparing metadata (setup.py) ... done


In [2]:
from urllib.request import urlretrieve
from pathlib import Path
import pandas as pd

In [3]:
from sbmlutils.io import read_sbml, validate_sbml
import networkx as nx
import py4cytoscape as p4c

In [4]:
from skm_tools import cytoscape_utils

## Download and load node (and reaction) annotations from SKM

In [5]:
PSS_NODE_URL = 'https://skm.nib.si/downloads/pss/public/sif-nodes'
pss_node_table_path = Path("./pss-node-annot.tsv")

if not pss_node_table_path.exists():
    print(f"Attempting to download the node annotations to {pss_node_table_path}.", end=" ")
    urlretrieve(PSS_NODE_URL, pss_node_table_path)
    print("Success.")

pss_node_df = pd.read_csv(pss_node_table_path, sep="\t")

In [6]:
pss_node_df.tail()

,name,node_type,short_name,functional_cluster_id,reaction_id,reaction_type,reaction_effect,additional_information,pathway,all_pathways,...,external_links,family,function,classification,components,ath_homologues,nta_homologues,osa_homologues,stu_homologues,sly_homologues
1553,MIZ1[AT2G41660],PlantCoding,MIZ1,fc00521,NaN,NaN,NaN,NaN,NaN,NaN,...,gmm:35.2,NaN,NaN,NaN,NaN,AT2G41660,NaN,NaN,NaN,NaN
1554,MIZ1|CPK6,Complex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"MIZ1[AT2G41660],CDPK[AT1G35670,AT2G17290,AT3G1...",NaN,NaN,NaN,NaN,NaN
1555,CPK12|GRF8,Complex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"GRF8[AT5G65430],CPK12[AT5G23580]",NaN,NaN,NaN,NaN,NaN
1556,CPK12[AT5G23580],PlantCoding,CPK12,fc00523,NaN,NaN,NaN,NaN,NaN,NaN,...,gmm:30.3,NaN,NaN,NaN,NaN,AT5G23580,NaN,NaN,NaN,NaN
1557,GRF8[AT5G65430],PlantCoding,GRF8,fc00522,NaN,NaN,NaN,NaN,NaN,NaN,...,gmm:30.7,NaN,NaN,NaN,NaN,AT5G65430,NaN,NaN,NaN,NaN


## Parse the SBML to a NetworkX DiGraph

In [7]:
sbml_path = Path("output.sbml")

In [8]:
doc = read_sbml(sbml_path)

In [9]:
model = doc.getModel()

In [10]:
g = nx.DiGraph()

In [11]:
for s in model.getListOfSpecies():
    parts = s.getId().split('_')
    label, loc, form = parts[1], parts[2], parts[3]
    g.add_node(s.id, db_name=s.getName(), label=label, sbml_compartment=s.getCompartment(), sbml_type='species', form=form)

In [12]:
for r in model.getListOfReactions():
    g.add_node(r.id, db_name=r.id, sbml_type='reaction', form='reaction')
    for m in r.getListOfModifiers():
        g.add_edge(m.getSpecies(), r.id, sbml_type="reaction-modifier")
    for s in r.getListOfReactants():
        g.add_edge(s.getSpecies(), r.id, sbml_type="reaction-reactant")
    for p in r.getListOfProducts():
        g.add_edge(r.id, p.getSpecies(), sbml_type="reaction-product")

In [13]:
g.number_of_nodes(), g.number_of_edges()

(1766, 2071)

## Add the node annotations to the graph

In [14]:
# first map graph nodes (SBML species) to the names in PSS
mapping  = [(n, data["db_name"]) for n, data in g.nodes(data=True)]
mapping_df = pd.DataFrame.from_records(mapping, columns=["id", "db_name"])
mapping_df.head()

,id,db_name
0,s_VPg_cyt_p,VPg
1,s_LHB1B1_cyt_p,LHB1B1[AT2G34430]
2,s_LHB1B1VPg_cyt_c,LHB1B1|VPg
3,s_PSB33_cyt_p,PSB33[AT1G71500]
4,s_PSB33VPg_cyt_c,PSB33|VPg


In [15]:
g_node_annots = mapping_df.join(pss_node_df.set_index("name"), on="db_name")
g_node_annots.tail()

,id,db_name,node_type,short_name,functional_cluster_id,reaction_id,reaction_type,reaction_effect,additional_information,pathway,...,external_links,family,function,classification,components,ath_homologues,nta_homologues,osa_homologues,stu_homologues,sly_homologues
1761,rx00960,rx00960,Reaction,NaN,NaN,rx00960,binding/oligomerisation,activation,NaN,NaN,...,doi:10.1016/j.molp.2023.04.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1762,rx00961,rx00961,Reaction,NaN,NaN,rx00961,translocation,activation,NaN,NaN,...,doi:10.1016/j.molp.2023.04.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1763,rx00962,rx00962,Reaction,NaN,NaN,rx00962,protein activation,activation,NaN,NaN,...,doi:10.1016/j.molp.2023.04.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1764,rx00963,rx00963,Reaction,NaN,NaN,rx00963,protein activation,activation,NaN,NaN,...,doi:10.1016/j.molp.2023.04.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1765,rx00964,rx00964,Reaction,NaN,NaN,rx00964,protein activation,activation,NaN,NaN,...,doi:10.1016/j.molp.2023.04.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
g_node_annots.set_index("id", inplace=True, drop=True)
nx.set_node_attributes(g, g_node_annots.to_dict('index'))

## View in Cytoscape

In [17]:
suid = p4c.create_network_from_networkx(g, title=str(sbml_path))
p4c.set_visual_style("sbml", suid) # this style is in the attached cytoscape session

Applying default style...
Applying preferred layout


{'message': 'Visual Style applied.'}

## Look for seperation of nodes based on "form" and/or "location"

In [18]:
# where the same node name appears more than once
multiple_occurances = [n 
                  for n, df 
                  in g_node_annots.groupby("db_name") 
                  if df.shape[0] > 1
]
print(len(multiple_occurances))

174


In [19]:
def extract_neighbourhood(nodes, graph):
    neighbours = set()
    for n in nodes:
        neighbours.update(graph.to_undirected(as_view=True).neighbors(n))
    return list(neighbours)

In [20]:
problematic_nodes = []
for target in multiple_occurances:
    sbml_species = [
        node
        for node, data
        in g.nodes(data=True)
        if data.get("db_name") == target
    ]
    
    reactions = extract_neighbourhood(sbml_species, g)
    subgraph_nodes = reactions + extract_neighbourhood(reactions, g)
    subgraph = nx.induced_subgraph(g, subgraph_nodes)
    is_species_connected = nx.is_weakly_connected(subgraph)

    if not is_species_connected:
        print(target, sbml_species, reactions)
        problematic_nodes.append(target)

AAO[AT1G04580,AT2G27150,AT5G20960] ['s_AAO_chl_pa', 's_AAO_cyt_pa'] ['rx00077', 'rx00552', 'rx00538']
ABA ['s_ABA_cyt_m', 's_ABA_ap_m'] ['rx00552', 'rx00749', 'rx00751', 'rx00474']
ACO[AT1G05010,AT1G12010,AT1G62380,AT1G77330,AT2G19590,SOTUB07G018820.1.1] ['s_ACO_cyt_pa', 's_ACO_er_pa', 's_ACO_cyt_p'] ['rx00193', 'rx00003', 'rx00855']
ACS[AT1G01480,AT2G22810,AT3G49700,AT3G61510,AT4G08040,AT4G11280,AT4G26200,AT4G37770,AT5G65800] ['s_ACS_cyt_pa', 's_ACS_cyt_p', 's_ACS_er_pa'] ['rx00716', 'rx00853', 'rx00140', 'rx00002', 'rx00248']
AGO1[AT1G48410] ['s_AGO1_cyt_pa', 's_AGO1_nuc_nc', 's_AGO1_cyt_mr'] ['rx00200', 'rx00199', 'rx00653', 'rx00589', 'rx00593']
AREB/ABF[AT1G45249,AT1G49720,AT3G19290,AT4G34000] ['s_AREBABF_cyt_p', 's_AREBABF_cyt_pa', 's_AREBABF_nuc_p'] ['rx00849', 'rx00576', 'rx00569', 'rx00607', 'rx00571', 'rx00848', 'rx00830', 'rx00605', 'rx00831', 'rx00610', 'rx00833', 'rx00832', 'rx00850']
ATAF1[AT1G01720] ['s_ATAF1_cyt_pa', 's_ATAF1_nuc_p'] ['rx00709', 'rx00710']
BA ['s_BA_chl

In [21]:
print(len(problematic_nodes))

85


In [22]:
# example to extract a "problematic" node in cytpscape
s_network_suid = cytoscape_utils.subnetwork_node_induced(
    subgraph_nodes,
    suid,
    name=target,
)
p4c.layout_network("cose", s_network_suid)

{}

## Extracting model input and output nodes

In [23]:
outputs = [node for node in g.nodes if g.out_degree(node) == 0 and node.startswith ('s_')]
outputs


[
    's_LHB1B1VPg_cyt_c',
    's_PSB33VPg_cyt_c',
    's_CRTHCPro_cyt_ca',
    's_EDF2HCPro_cyt_ca',
    's_CMLHCPro_cyt_ca',
    's_EDS1MPK3PAD4_cyt_ca',
    's_SERPIN1VPg_cyt_c',
    's_CLPP1VPg_cyt_c',
    's_FQR1VPg_cyt_c',
    's_PR3VPg_cyt_c',
    's_HCProTDX_cyt_ca',
    's_P3RBC_cyt_ca',
    's_OBE1VPg_cyt_ca',
    's_RH8VPg_cyt_ca',
    's_RPS12CVPg_cyt_c',
    's_MTI2011VPg_cyt_c',
    's_CRTETR_cyt_ca',
    's_CRTCa2_cyt_ca',
    's_SAMS_er_p',
    's_SAHH_er_p',
    's_HSP90RAR1SGT1_cyt_ca',
    's_PR5VPg_cyt_c',
    's_AGO15710HCPro_cyt_ca',
    's_AGO15710CI_cyt_ca',
    's_CPCPIP_cyt_ca',
    's_ATG13A_cyt_p',
    's_DXPS2HCPro_cyt_ca',
    's_ATPBHCPro_cyt_ca',
    's_NDR1RIN4_cyt_ca',
    's_HCProMIND1_cyt_ca',
    's_HCProPAA2_cyt_ca',
    's_HCProPBB2_cyt_ca',
    's_RANGAPRx_cyt_ca',
    's_GPAphid2RANGAP_cyt_ca',
    's_HCProPBE1_cyt_ca',
    's_OBE1WRKY17_cyt_ca',
    's_OBE1WRKY11_cyt_ca',
    's_COOBE1_cyt_ca',
    's_COI1RBC_cyt_ca',
    's_JAZRBC_cyt_ca',
  

In [24]:
inputs = [node for node in g.nodes if g.in_degree(node) == 0 and node.startswith ('s_')]
inputs


[
    's_VPg_cyt_p',
    's_LHB1B1_cyt_p',
    's_PSB33_cyt_p',
    's_HCPro_cyt_p',
    's_OSCA1_cyt_p',
    's_Drought_cyt_ab',
    's_EDF2_cyt_p',
    's_CML_cyt_pa',
    's_BZO1_cyt_pa',
    's_SERPIN1_cyt_p',
    's_clpP1_cyt_p',
    's_FQR1_cyt_p',
    's_PR3like_cyt_p',
    's_TDX_cyt_p',
    's_P3_cyt_p',
    's_RBC_cyt_pa',
    's_OBE1_cyt_p',
    's_RH8_cyt_p',
    's_RPS12C_cyt_p',
    's_MTI2011_cyt_p',
    's_ETR_er_p',
    's_SAMS_er_pa',
    's_SAHH_er_pa',
    's_HSP90_cyt_p',
    's_RAR1_cyt_p',
    's_SGT1_cyt_p',
    's_PR5_cyt_p',
    's_AGO1_cyt_pa',
    's_SAGs_nuc_p',
    's_CI_cyt_p',
    's_CPIP_cyt_pa',
    's_CP_cyt_p',
    's_ATG13A_cyt_pa',
    's_ATPB_cyt_p',
    's_NDR1_cyt_p',
    's_MIND1_cyt_p',
    's_PAA2_cyt_p',
    's_PBB2_cyt_p',
    's_TGA256_nuc_pa',
    's_D14_cyt_p',
    's_Rx_cyt_p',
    's_RANGAP_cyt_pa',
    's_GPAphid2_cyt_p',
    's_PBE1_cyt_p',
    's_WRKY17_cyt_p',
    's_WRKY11_cyt_p',
    's_GAPC2_cyt_p',
    's_AT3G17020_cyt_p',
   